## **Raport**

- **mat_multiply.rs** – plik, w którym znajduje się implementacja mnożenia macierzy  
- **constraints.rs** – zawiera struktury i funkcje ułatwiające operacje na macierzach
- **validation.rs** – zawiera walidację macierzy (reguły dotyczące rozmiarów macierzy do mnożenia)


### **Przykład mnożenia macierzy** 
(wykorzystanie Pythona poprzez Binding z Rust do Pythona)

In [ ]:
import matrix_rs

A = [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16]]

B = [[16, 15, 14, 13], [12, 11, 10, 9], [8, 7, 6, 5], [4, 3, 2, 1]]

result = matrix_rs.mul(A, B)

print("Result:")
for row in result:
    print(row)



## **1. Funkcja multiply()**  

1. Funkcja przyjmuje dwie macierze `a` i `b` w formacie `Vec<Vec<f64>>` (wektor wektorow liczb zmiennoprzecinkowych).  
   Zwraca `Result<Vec<Vec<f64>>, MatricesError>`, czyli:  
   - **`Ok(Vec<Vec<f64>>)`** – jeśli operacja się powiedzie, zwraca wynikową macierz.  
   - **`Err(MatricesError)`** – jeśli wystąpi błąd, np. macierze są niepoprawne (nie są kwadratowe, mają niewłaściwe rozmiary).  
```rust
        pub fn multiply(a: Vec<Vec<f64>>, b: Vec<Vec<f64>>) -> Result<Vec<Vec<f64>>, MatricesError> {
```
2. **Walidacja macierzy** 
    - Czy macierze są kwadratowe?
    - Czy ich rozmiary są potęgami dwójki?
    - Czy macierze mają ten sam rozmiar? 
```rust
        validate_matrices(&a, &b)?;
```
3. **Tworzenie offsetów**  
   - Określenie **początkowego wiersza, początkowej kolumny i rozmiaru**.  
   - Offsety pozwalają na łatwe operowanie na podmacierzach bez kopiowania danych.  
   ```rust
        let a_offset = Offset::new_unchecked(&a);
        let b_offset = Offset::new_unchecked(&b);
   ```
4. **Inicjalizacja macierzy wynikowej `c`**  
```rust
        let n = a_offset.n;
        let mut c = vec![vec![0.0; n]; n];
        let c_offset = Offset::new_unchecked(&c);
```
5. **Wywołanie funkcji `multiply_into`** (rekurencja)  
   - **Rekurencyjne dzielenie macierzy na podmacierze**.  
   - **Dla macierzy o rozmiarze ≤ 2x2 używa klasycznej metody mnożenia**.
```rust
         multiply_into(
            &a_offset.as_matrix_with_offset(&a),
            &b_offset.as_matrix_with_offset(&b),
            &mut c_offset.as_matrix_with_offset_mut(&mut c),
            );`
```
6. **Zwracanie wyniku**  
```rust
        Ok(c)
        
```



```rust

pub fn multiply(a: Vec<Vec<f64>>, b: Vec<Vec<f64>>) -> Result<Vec<Vec<f64>>, MatricesError> {

    validate_matrices(&a, &b)?;
    
    let a_offset = Offset::new_unchecked(&a);
    let b_offset = Offset::new_unchecked(&b);

    let n = a_offset.n;
    let mut c = vec![vec![0.0; n]; n];
    let c_offset = Offset::new_unchecked(&c);

    multiply_into(
        &a_offset.as_matrix_with_offset(&a),
        &b_offset.as_matrix_with_offset(&b),
        &mut c_offset.as_matrix_with_offset_mut(&mut c),
    );
    Ok(c)
}
```

## Funkcja `multiply_into()`
Funkcja `multiply_into` rekurencyjnie mnoży dwie macierze `a` i `b`, a wynik zapisuje do macierzy `c`.

---


1. **Sprawdzenie rozmiaru macierzy oraz wykonanie klasycznego mnożenia dla odpoweidniego rozmiaru**
   ```rust
   fn multiply_into(a: &MatrixWithOffset, b: &MatrixWithOffset, c: &mut MatrixWithOffsetMut) {
        let n = a.offset.n;
        if n <= THRESHOLD {
            naive_multiply_into(a, b, c);
            return;
        }
2. **Obliczenie połowy rozmiaru macierzy**
```rust
    let mid = n / 2;
```
3. **Podział macierzy a,b i c na 4 podmacierze**
```rust
    let split = a.offset.split();
    let Split { s11: a11, s12: a12, s21: a21, s22: a22 } = a.create_from_split(&split);
```
4. **Utworzenie bufora pomocniczego** (Przechowuje częściowe wyniki mnożenia)
```rust
    let mut buffer = vec![vec![0.0; mid]; mid];
    let buffer_offset = Offset::new_unchecked(&buffer);
```
5. **Mnożenie i dodawanie kolejnych podmacierzy (rekurencja multiply_into() oraz add_into()): c11 = a11 * b11 + a12 * b21**
```rust
    // upper left
    multiply_into(&a11, &b11, &mut buffer_offset.as_matrix_with_offset_mut(&mut buffer));
    multiply_into(&a12, &b21, &mut c11_offset.as_matrix_with_offset_mut(c.matrix));
    add_into(&mut c11_offset.as_matrix_with_offset_mut(c.matrix), &buffer_offset.as_matrix_with_offset(&buffer));
    // upper right
    multiply_into(&a11, &b12, &mut buffer_offset.as_matrix_with_offset_mut(&mut buffer));
    multiply_into(&a12, &b22, &mut c12_offset.as_matrix_with_offset_mut(c.matrix));
    add_into(&mut c12_offset.as_matrix_with_offset_mut(c.matrix), &buffer_offset.as_matrix_with_offset(&buffer));
    // lower left
    multiply_into(&a21, &b11, &mut buffer_offset.as_matrix_with_offset_mut(&mut buffer));
    multiply_into(&a22, &b21, &mut c21_offset.as_matrix_with_offset_mut(c.matrix));
    add_into(&mut c21_offset.as_matrix_with_offset_mut(c.matrix), &buffer_offset.as_matrix_with_offset(&buffer));
    // lower right
    multiply_into(&a21, &b12, &mut buffer_offset.as_matrix_with_offset_mut(&mut buffer));
    multiply_into(&a22, &b22, &mut c22_offset.as_matrix_with_offset_mut(c.matrix));
    add_into(&mut c22_offset.as_matrix_with_offset_mut(c.matrix), &buffer_offset.as_matrix_with_offset(&buffer));
```






## Funkcja `add_into`() oraz naive_multiply_into()

Funkcja `add_into` dodaje odpowiadające sobie elementy macierzy `b` do macierzy `a`.

---

**Rozmiar podmacierzy, Iteracja po wszystkich wierszach i kolumnach, Dodanie wartości z b do a** 
```rust
    fn add_into(a: &mut MatrixWithOffsetMut, b: &MatrixWithOffset) {
        let n = b.offset.n;

        for i in 0..n {
            for j in 0..n {
                a[(i, j)] += b[(i, j)];
            }
        }
    }
```
---
**Klasyczne mnożenie macierzy:** 
1. Rozmiar macierzy, 
2. Iteracja po wierszach wynikowej macierzy, 
3. Iteracja po kolumnach wynikowej macierzy, 
4. Zainicjalizowanie elementu wyniku zerem, Pętla sumująca iloczyny elementów

```rust
    pub fn naive_multiply_into(
        a: &MatrixWithOffset,
        b: &MatrixWithOffset,
        result: &mut MatrixWithOffsetMut,
    ) {
        let n = a.offset.n;
        for i in 0..n {
            for j in 0..n {
                result[(i, j)] = 0.0;
                for k in 0..n {
                    result[(i, j)] += a[(i, k)] * b[(k, j)];
                }
            }
        }
    }
```

